In [75]:
from datasets.karel import parser_for_synthesis, mutation, karel_runtime
from datasets import dataset, executor

import collections
import copy
import json
import cPickle as pickle
import pprint

import numpy as np

In [34]:
reload(karel_runtime)

<module 'datasets.karel.karel_runtime' from 'datasets/karel/karel_runtime.py'>

In [78]:
def visualize_trace(trace):
    kr = karel_runtime.KarelRuntime()
    event_idx = 0
    for i, grid in enumerate(trace.grids):
        while event_idx < len(trace.events) and trace.events[event_idx].timestep == i:
            print trace.events[event_idx]
            event_idx += 1
        field = np.zeros((15, 18, 18), dtype=np.bool)
        field.ravel()[grid] = True
        kr.init_from_array(field)
        kr.draw()
    while event_idx < len(trace.events):
            print trace.events[event_idx]
            event_idx += 1

def vis(i, j, code=None):
    result = the_executor.execute(
            code if code else ds[i].code_sequence, None, ds[i].input_tests[j]['input'], record_trace=True, strict=True)
    visualize_trace(result.trace)
            
the_executor = executor.KarelExecutor()

In [51]:
with open('data/karel/train.pkl') as f:
    while True:
        example = pickle.load(f)
        if example['guid'] == '226540459e33b9d9':
            break

In [52]:
' '.join(example['code'])

u'DEF run m( turnRight move turnLeft WHILE c( markersPresent c) w( putMarker REPEAT R=5 r( IFELSE c( leftIsClear c) i( WHILE c( noMarkersPresent c) w( putMarker w) i) ELSE e( putMarker e) move r) w) m)'

In [53]:
mutated_code = '''DEF run m(
    turnRight
    move
    turnLeft
    WHILE c( markersPresent c) w(
        putMarker
        REPEAT R=5 r(
            turnLeft
            IFELSE c( leftIsClear c) i(
                WHILE c( noMarkersPresent c) w(
                    putMarker
                w)
            i) ELSE e(
                putMarker
            e)
            move
        r)
    w)
m)'''

In [54]:
result = the_executor.execute(mutated_code.split(), None, example['examples'][88 % 5]['in'], record_trace=True, strict=True)

In [55]:
result.trace.events

[KarelEvent(timestep=1, type='turnRight', span=(3, 3), cond_span=None, cond_value=None, success=True),
 KarelEvent(timestep=2, type='move', span=(4, 4), cond_span=None, cond_value=None, success=True),
 KarelEvent(timestep=3, type='turnLeft', span=(5, 5), cond_span=None, cond_value=None, success=True),
 KarelEvent(timestep=4, type='while', span=(6, 35), cond_span=(8, 8), cond_value=True, success=True),
 KarelEvent(timestep=4, type='putMarker', span=(11, 11), cond_span=None, cond_value=None, success=True),
 KarelEvent(timestep=5, type='repeat', span=(12, 34), cond_span=(13, 13), cond_value=5, success=True),
 KarelEvent(timestep=5, type='turnLeft', span=(15, 15), cond_span=None, cond_value=None, success=True),
 KarelEvent(timestep=6, type='ifElse', span=(16, 32), cond_span=(18, 18), cond_value=True, success=True),
 KarelEvent(timestep=6, type='while', span=(21, 27), cond_span=(23, 23), cond_value=False, success=True),
 KarelEvent(timestep=6, type='move', span=(33, 33), cond_span=None, con

In [56]:
orig_result = the_executor.execute(example['code'], None, example['examples'][88 % 5]['in'], record_trace=True, strict=True)
visualize_trace(orig_result.trace)

█████████████████
█               █
█  1            █
█ >             █
█ 1   1        1█
█████████████████
KarelEvent(timestep=1, type='turnRight', span=(3, 3), cond_span=None, cond_value=None, success=True)
█████████████████
█               █
█  1            █
█ ↓             █
█ 1   1        1█
█████████████████
KarelEvent(timestep=2, type='move', span=(4, 4), cond_span=None, cond_value=None, success=True)
█████████████████
█               █
█  1            █
█               █
█ 1̲   1        1█
█████████████████
KarelEvent(timestep=3, type='turnLeft', span=(5, 5), cond_span=None, cond_value=None, success=True)
█████████████████
█               █
█  1            █
█               █
█ 1͕   1        1█
█████████████████
KarelEvent(timestep=4, type='while', span=(6, 34), cond_span=(8, 8), cond_value=True, success=True)
KarelEvent(timestep=4, type='putMarker', span=(11, 11), cond_span=None, cond_value=None, success=True)
█████████████████
█               █
█  1            █
█           

█████████████████
█               █
█  1            █
█ >             █
█ 1   1        1█
█████████████████
KarelEvent(timestep=1, type='turnRight', span=(3, 3), cond_span=None, cond_value=None, success=True)
█████████████████
█               █
█  1            █
█ ↓             █
█ 1   1        1█
█████████████████
KarelEvent(timestep=2, type='move', span=(4, 4), cond_span=None, cond_value=None, success=True)
█████████████████
█               █
█  1            █
█               █
█ 1̲   1        1█
█████████████████
KarelEvent(timestep=3, type='turnLeft', span=(5, 5), cond_span=None, cond_value=None, success=True)
█████████████████
█               █
█  1            █
█               █
█ 1͕   1        1█
█████████████████
KarelEvent(timestep=4, type='while', span=(6, 34), cond_span=(8, 8), cond_value=True, success=True)
KarelEvent(timestep=4, type='putMarker', span=(11, 11), cond_span=None, cond_value=None, success=True)
█████████████████
█               █
█  1            █
█           

In [35]:
visualize_trace(result.trace)

█████████████████
█               █
█  1            █
█ >             █
█ 1   1        1█
█████████████████
KarelEvent(timestep=1, type='turnRight', span=(3, 3), cond_span=None, cond_value=None, success=True)
█████████████████
█               █
█  1            █
█ ↓             █
█ 1   1        1█
█████████████████
KarelEvent(timestep=2, type='move', span=(4, 4), cond_span=None, cond_value=None, success=True)
█████████████████
█               █
█  1            █
█               █
█ 1̲   1        1█
█████████████████
KarelEvent(timestep=3, type='turnLeft', span=(5, 5), cond_span=None, cond_value=None, success=True)
█████████████████
█               █
█  1            █
█               █
█ 1͕   1        1█
█████████████████
KarelEvent(timestep=4, type='while', span=(6, 35), cond_span=(8, 8), cond_value=True, success=True)
KarelEvent(timestep=4, type='putMarker', span=(11, 11), cond_span=None, cond_value=None, success=True)
█████████████████
█               █
█  1            █
█           

█77             █
█77͕   1        1█
█████████████████
KarelEvent(timestep=271, type='repeat', span=(12, 34), cond_span=(13, 13), cond_value=0, success=True)
KarelEvent(timestep=271, type='while', span=(6, 35), cond_span=(8, 8), cond_value=True, success=True)
KarelEvent(timestep=271, type='putMarker', span=(11, 11), cond_span=None, cond_value=None, success=True)
█████████████████
█               █
█  1            █
█77             █
█78͕   1        1█
█████████████████
KarelEvent(timestep=272, type='repeat', span=(12, 34), cond_span=(13, 13), cond_value=5, success=True)
KarelEvent(timestep=272, type='turnLeft', span=(15, 15), cond_span=None, cond_value=None, success=True)
█████████████████
█               █
█  1            █
█77             █
█78̅   1        1█
█████████████████
KarelEvent(timestep=273, type='ifElse', span=(16, 32), cond_span=(18, 18), cond_value=True, success=True)
KarelEvent(timestep=273, type='while', span=(21, 27), cond_span=(23, 23), cond_value=False, success=True

In [64]:
with open('data/karel/train.pkl') as f:
    while True:
        example = pickle.load(f)
        if example['guid'] == 'a2b3797eeb772561':
            break

In [65]:
' '.join(example['code'])

u'DEF run m( WHILE c( markersPresent c) w( move REPEAT R=2 r( move putMarker IFELSE c( frontIsClear c) i( pickMarker i) ELSE e( turnRight e) r) w) m)'

In [66]:
mutated_code = ('DEF', 'run', 'm(', 'WHILE', 'c(', 'markersPresent', 'c)', 'w(', 'move', 'REPEAT', 'R=2', 'r(', 'putMarker', 'putMarker', 'IFELSE', 'c(', 'frontIsClear', 'c)', 'i(', 'pickMarker', 'i)', 'ELSE', 'e(',
'turnRight', 'e)', 'r)', 'w)', 'm)')
' '.join(mutated_code)

'DEF run m( WHILE c( markersPresent c) w( move REPEAT R=2 r( putMarker putMarker IFELSE c( frontIsClear c) i( pickMarker i) ELSE e( turnRight e) r) w) m)'

In [67]:
orig_result = the_executor.execute(
    example['code'], None, example['examples'][65 % 5]['in'], record_trace=True, strict=True)

In [68]:
visualize_trace(orig_result.trace)

█████████████████
█ 1         ░   █
█     1   ░     █
█ 11  1      ░░░█
█ 1            ░█
█  ░       ░    █
█1              █
█░  1͔   1       █
█        5 ░3   █
█     ░1  ░     █
█               █
█          1    █
█       ░74   1 █
█████████████████
KarelEvent(timestep=1, type='while', span=(3, 26), cond_span=(5, 5), cond_value=True, success=True)
KarelEvent(timestep=1, type='move', span=(8, 8), cond_span=None, cond_value=None, success=True)
█████████████████
█ 1         ░   █
█     1   ░     █
█ 11  1      ░░░█
█ 1            ░█
█  ░       ░    █
█1              █
█░ <1   1       █
█        5 ░3   █
█     ░1  ░     █
█               █
█          1    █
█       ░74   1 █
█████████████████
KarelEvent(timestep=2, type='repeat', span=(9, 25), cond_span=(10, 10), cond_value=2, success=True)
KarelEvent(timestep=2, type='move', span=(12, 12), cond_span=None, cond_value=None, success=True)
█████████████████
█ 1         ░   █
█     1   ░     █
█ 11  1      ░░░█
█ 1            ░█
█  ░       

In [69]:
mut_result = the_executor.execute(
    mutated_code, None, example['examples'][65 % 5]['in'], record_trace=True, strict=True)

In [70]:
visualize_trace(mut_result.trace)

█████████████████
█ 1         ░   █
█     1   ░     █
█ 11  1      ░░░█
█ 1            ░█
█  ░       ░    █
█1              █
█░  1͔   1       █
█        5 ░3   █
█     ░1  ░     █
█               █
█          1    █
█       ░74   1 █
█████████████████
KarelEvent(timestep=1, type='while', span=(3, 26), cond_span=(5, 5), cond_value=True, success=True)
KarelEvent(timestep=1, type='move', span=(8, 8), cond_span=None, cond_value=None, success=True)
█████████████████
█ 1         ░   █
█     1   ░     █
█ 11  1      ░░░█
█ 1            ░█
█  ░       ░    █
█1              █
█░ <1   1       █
█        5 ░3   █
█     ░1  ░     █
█               █
█          1    █
█       ░74   1 █
█████████████████
KarelEvent(timestep=2, type='repeat', span=(9, 25), cond_span=(10, 10), cond_value=2, success=True)
KarelEvent(timestep=2, type='putMarker', span=(12, 12), cond_span=None, cond_value=None, success=True)
█████████████████
█ 1         ░   █
█     1   ░     █
█ 11  1      ░░░█
█ 1            ░█
█  ░  

█ 4222222222͕░   █
█ 2   1   ░     █
█ 31  1      ░░░█
█ 3            ░█
█ 2░       ░    █
█12             █
█░321   1       █
█        5 ░3   █
█     ░1  ░     █
█               █
█          1    █
█       ░74   1 █
█████████████████
KarelEvent(timestep=116, type='ifElse', span=(14, 24), cond_span=(16, 16), cond_value=False, success=True)
KarelEvent(timestep=116, type='turnRight', span=(23, 23), cond_span=None, cond_value=None, success=True)
█████████████████
█ 4222222222̲░   █
█ 2   1   ░     █
█ 31  1      ░░░█
█ 3            ░█
█ 2░       ░    █
█12             █
█░321   1       █
█        5 ░3   █
█     ░1  ░     █
█               █
█          1    █
█       ░74   1 █
█████████████████
KarelEvent(timestep=117, type='repeat', span=(9, 25), cond_span=(10, 10), cond_value=1, success=True)
KarelEvent(timestep=117, type='putMarker', span=(12, 12), cond_span=None, cond_value=None, success=True)
█████████████████
█ 4222222223̲░   █
█ 2   1   ░     █
█ 31  1      ░░░█
█ 3            ░█
█ 

█               █
█          1    █
█       ░74   1 █
█████████████████
KarelEvent(timestep=263, type='ifElse', span=(14, 24), cond_span=(16, 16), cond_value=True, success=True)
KarelEvent(timestep=263, type='pickMarker', span=(19, 19), cond_span=None, cond_value=None, success=True)
█████████████████
█364443͕22223░   █
█22   1   ░2    █
█231  1    2 ░░░█
█35222222223   ░█
█ 2░       ░    █
█12             █
█░321   1       █
█        5 ░3   █
█     ░1  ░     █
█               █
█          1    █
█       ░74   1 █
█████████████████
KarelEvent(timestep=264, type='repeat', span=(9, 25), cond_span=(10, 10), cond_value=1, success=True)
KarelEvent(timestep=264, type='putMarker', span=(12, 12), cond_span=None, cond_value=None, success=True)
█████████████████
█364444͕22223░   █
█22   1   ░2    █
█231  1    2 ░░░█
█35222222223   ░█
█ 2░       ░    █
█12             █
█░321   1       █
█        5 ░3   █
█     ░1  ░     █
█               █
█          1    █
█       ░74   1 █
█████████████████
Kar

█████████████████
KarelEvent(timestep=452, type='ifElse', span=(14, 24), cond_span=(16, 16), cond_value=True, success=True)
KarelEvent(timestep=452, type='pickMarker', span=(19, 19), cond_span=None, cond_value=None, success=True)
█████████████████
█6866665͕4446░   █
█42   1   ░4    █
█431  1    4 ░░░█
█67444444446   ░█
█ 2░       ░    █
█12             █
█░321   1       █
█        5 ░3   █
█     ░1  ░     █
█               █
█          1    █
█       ░74   1 █
█████████████████
KarelEvent(timestep=453, type='repeat', span=(9, 25), cond_span=(10, 10), cond_value=1, success=True)
KarelEvent(timestep=453, type='putMarker', span=(12, 12), cond_span=None, cond_value=None, success=True)
█████████████████
█6866666͕4446░   █
█42   1   ░4    █
█431  1    4 ░░░█
█67444444446   ░█
█ 2░       ░    █
█12             █
█░321   1       █
█        5 ░3   █
█     ░1  ░     █
█               █
█          1    █
█       ░74   1 █
█████████████████
KarelEvent(timestep=454, type='putMarker', span=(13, 13),

In [71]:
mut_result.trace.events[-5:]

[KarelEvent(timestep=480, type='ifElse', span=(14, 24), cond_span=(16, 16), cond_value=False, success=True),
 KarelEvent(timestep=480, type='turnRight', span=(23, 23), cond_span=None, cond_value=None, success=True),
 KarelEvent(timestep=481, type='repeat', span=(9, 25), cond_span=(10, 10), cond_value=1, success=True),
 KarelEvent(timestep=481, type='putMarker', span=(12, 12), cond_span=None, cond_value=None, success=True),
 KarelEvent(timestep=482, type='putMarker', span=(13, 13), cond_span=None, cond_value=None, success=False)]

In [72]:
orig_result_2 = the_executor.execute(
    example['code'], None, example['examples'][67 % 5]['in'], record_trace=True, strict=True)
visualize_trace(orig_result_2.trace)

████████████████
█  ░░ 1  2░ 1  █
█              █
█3          ░  █
█       ░ ░  ░ █
█  ░  11  4░1░░█
█2̅         ░   █
████████████████
KarelEvent(timestep=1, type='while', span=(3, 26), cond_span=(5, 5), cond_value=True, success=True)
KarelEvent(timestep=1, type='move', span=(8, 8), cond_span=None, cond_value=None, success=True)
████████████████
█  ░░ 1  2░ 1  █
█              █
█3          ░  █
█       ░ ░  ░ █
█↑ ░  11  4░1░░█
█2         ░   █
████████████████
KarelEvent(timestep=2, type='repeat', span=(9, 25), cond_span=(10, 10), cond_value=2, success=True)
KarelEvent(timestep=2, type='move', span=(12, 12), cond_span=None, cond_value=None, success=True)
████████████████
█  ░░ 1  2░ 1  █
█              █
█3          ░  █
█↑      ░ ░  ░ █
█  ░  11  4░1░░█
█2         ░   █
████████████████
KarelEvent(timestep=3, type='putMarker', span=(13, 13), cond_span=None, cond_value=None, success=True)
████████████████
█  ░░ 1  2░ 1  █
█              █
█3          ░  █
█1̅      ░ ░  ░ █
█  ░  11

In [73]:
mut_result_2 = the_executor.execute(
    mutated_code, None, example['examples'][67 % 5]['in'], record_trace=True, strict=True)
visualize_trace(mut_result_2.trace)

████████████████
█  ░░ 1  2░ 1  █
█              █
█3          ░  █
█       ░ ░  ░ █
█  ░  11  4░1░░█
█2̅         ░   █
████████████████
KarelEvent(timestep=1, type='while', span=(3, 26), cond_span=(5, 5), cond_value=True, success=True)
KarelEvent(timestep=1, type='move', span=(8, 8), cond_span=None, cond_value=None, success=True)
████████████████
█  ░░ 1  2░ 1  █
█              █
█3          ░  █
█       ░ ░  ░ █
█↑ ░  11  4░1░░█
█2         ░   █
████████████████
KarelEvent(timestep=2, type='repeat', span=(9, 25), cond_span=(10, 10), cond_value=2, success=True)
KarelEvent(timestep=2, type='putMarker', span=(12, 12), cond_span=None, cond_value=None, success=True)
████████████████
█  ░░ 1  2░ 1  █
█              █
█3          ░  █
█       ░ ░  ░ █
█1̅ ░  11  4░1░░█
█2         ░   █
████████████████
KarelEvent(timestep=3, type='putMarker', span=(13, 13), cond_span=None, cond_value=None, success=True)
████████████████
█  ░░ 1  2░ 1  █
█              █
█3          ░  █
█       ░ ░  ░ █
█2̅

In [76]:
ds = dataset.KarelTorchDataset('data/karel/train.pkl')

In [80]:
vis(9001, 0)

████████
█   ░  █
█    ░ █
█ ░    █
█  >   █
█ ░    █
█    ░ █
█  ░  1█
█ ░  1░█
█░     █
█      █
█   11 █
█8     █
████████
KarelEvent(timestep=1, type='move', span=(3, 3), cond_span=None, cond_value=None, success=True)
████████
█   ░  █
█    ░ █
█ ░    █
█   >  █
█ ░    █
█    ░ █
█  ░  1█
█ ░  1░█
█░     █
█      █
█   11 █
█8     █
████████
KarelEvent(timestep=2, type='repeat', span=(4, 49), cond_span=(5, 5), cond_value=5, success=True)
KarelEvent(timestep=2, type='while', span=(7, 41), cond_span=(9, 9), cond_value=True, success=True)
KarelEvent(timestep=2, type='putMarker', span=(12, 12), cond_span=None, cond_value=None, success=True)
████████
█   ░  █
█    ░ █
█ ░    █
█   1͕  █
█ ░    █
█    ░ █
█  ░  1█
█ ░  1░█
█░     █
█      █
█   11 █
█8     █
████████
KarelEvent(timestep=3, type='while', span=(13, 40), cond_span=(15, 15), cond_value=True, success=True)
KarelEvent(timestep=3, type='ifElse', span=(18, 28), cond_span=(20, 20), cond_value=True, success=True)
KarelEvent(timest

In [81]:
mut_code = ('DEF', 'run', 'm(', 'move', 'REPEAT', 'R=5', 'r(', 'WHILE', 'c(', 'leftIsClear', 'c)', 'w(', 'putMarker', 'WHILE', 'c(', 'leftIsClear', 'c)', 'w(', 'IFELSE', 'c(', 'frontIsClear', 'c)', 'i(', 'move',
'putMarker', 'i)', 'ELSE', 'e(', 'putMarker', 'e)', 'REPEAT', 'R=6', 'r(', 'REPEAT', 'R=3', 'r(', 'turnLeft', 'r)', 'r)', 'move', 'move', 'w)', 'w)', 'IF', 'c(', 'rightIsClear', 'c)', 'i(', 'pickMarker',
'i)', 'r)', 'm)')
vis(9001, 0, mut_code)

████████
█   ░  █
█    ░ █
█ ░    █
█  >   █
█ ░    █
█    ░ █
█  ░  1█
█ ░  1░█
█░     █
█      █
█   11 █
█8     █
████████
KarelEvent(timestep=1, type='move', span=(3, 3), cond_span=None, cond_value=None, success=True)
████████
█   ░  █
█    ░ █
█ ░    █
█   >  █
█ ░    █
█    ░ █
█  ░  1█
█ ░  1░█
█░     █
█      █
█   11 █
█8     █
████████
KarelEvent(timestep=2, type='repeat', span=(4, 50), cond_span=(5, 5), cond_value=5, success=True)
KarelEvent(timestep=2, type='while', span=(7, 42), cond_span=(9, 9), cond_value=True, success=True)
KarelEvent(timestep=2, type='putMarker', span=(12, 12), cond_span=None, cond_value=None, success=True)
████████
█   ░  █
█    ░ █
█ ░    █
█   1͕  █
█ ░    █
█    ░ █
█  ░  1█
█ ░  1░█
█░     █
█      █
█   11 █
█8     █
████████
KarelEvent(timestep=3, type='while', span=(13, 41), cond_span=(15, 15), cond_value=True, success=True)
KarelEvent(timestep=3, type='ifElse', span=(18, 29), cond_span=(20, 20), cond_value=True, success=True)
KarelEvent(timest

█ ░    █
█ 4 15̲ █
█ ░    █
█    ░ █
█  ░  1█
█ ░  1░█
█░     █
█      █
█   11 █
█8     █
████████
KarelEvent(timestep=196, type='repeat', span=(33, 37), cond_span=(34, 34), cond_value=0, success=True)
KarelEvent(timestep=196, type='repeat', span=(30, 38), cond_span=(31, 31), cond_value=1, success=True)
KarelEvent(timestep=196, type='repeat', span=(33, 37), cond_span=(34, 34), cond_value=3, success=True)
KarelEvent(timestep=196, type='turnLeft', span=(36, 36), cond_span=None, cond_value=None, success=True)
████████
█   ░  █
█    ░ █
█ ░    █
█ 4 15͕ █
█ ░    █
█    ░ █
█  ░  1█
█ ░  1░█
█░     █
█      █
█   11 █
█8     █
████████
KarelEvent(timestep=197, type='repeat', span=(33, 37), cond_span=(34, 34), cond_value=2, success=True)
KarelEvent(timestep=197, type='turnLeft', span=(36, 36), cond_span=None, cond_value=None, success=True)
████████
█   ░  █
█    ░ █
█ ░    █
█ 4 15̅ █
█ ░    █
█    ░ █
█  ░  1█
█ ░  1░█
█░     █
█      █
█   11 █
█8     █
████████
KarelEvent(timestep=198, t

KarelEvent(timestep=375, type='repeat', span=(33, 37), cond_span=(34, 34), cond_value=0, success=True)
KarelEvent(timestep=375, type='repeat', span=(30, 38), cond_span=(31, 31), cond_value=0, success=True)
KarelEvent(timestep=375, type='move', span=(39, 39), cond_span=None, cond_value=None, success=True)
████████
█   ░  █
█    ░ █
█ ░    █
█ 8 1͔9 █
█ ░    █
█    ░ █
█  ░  1█
█ ░  1░█
█░     █
█      █
█   11 █
█8     █
████████
KarelEvent(timestep=376, type='move', span=(40, 40), cond_span=None, cond_value=None, success=True)
████████
█   ░  █
█    ░ █
█ ░    █
█ 8<19 █
█ ░    █
█    ░ █
█  ░  1█
█ ░  1░█
█░     █
█      █
█   11 █
█8     █
████████
KarelEvent(timestep=377, type='while', span=(13, 41), cond_span=(15, 15), cond_value=True, success=True)
KarelEvent(timestep=377, type='ifElse', span=(18, 29), cond_span=(20, 20), cond_value=True, success=True)
KarelEvent(timestep=377, type='move', span=(23, 23), cond_span=None, cond_value=None, success=True)
████████
█   ░  █
█    ░ █
█ ░